In [2]:
from pathlib import Path
import numpy as np

import sys
sys.path.append("..")
import ICE.utils

sys.path.append("/share/home/francesco/concept_composers/DeepComposerClassification/")
sys.path.append("/share/home/francesco/concept_composers/")

utils = ICE.utils.img_utils()

In [7]:
mozart_path = Path("/share/cp/projects/concept_composers/experiments/kim2020/npy/composer12")

for folder in mozart_path.iterdir():
    if len(list(folder.iterdir())) != 1:
        print(folder.name)

midi18
midi20
midi27
midi29


In [20]:
for folder in mozart_path.iterdir():
    for piece in folder.iterdir():
        out_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check",folder.name + piece.name + ".mid")
        pr = np.transpose(np.load(piece), (0, 2, 1))
        utils.pianoroll2midi(pr, out_path, samples_per_second=20)


# Try with different fs

In [2]:
from data_handling.io import open_midi
from DeepComposerClassification.generator import Generator

In [3]:
generator = Generator()

def midi2pianoroll(midi_path, out_npy_path, frame_length):
    x = open_midi(str(midi_path))
    x = generator.generate_segment(x, frame_length = frame_length)
    x = np.array(x)
    np.save(str(out_npy_path), x)

In [57]:
midi_path = Path("/share/cp/projects/concept_composers/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_01_R1_2015_wav--3.midi")
npy_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/pianoroll_ex.npy")
recreated_midi_path = ("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/new_midi.mid")

midi2pianoroll(midi_path,npy_path, frame_length = 0.05)

generating... dur: 230.00sec || len: 4600
Shape (2, 4600, 128)
1
104


In [8]:
pr = np.transpose(np.load(npy_path), (0, 2, 1))
utils.pianoroll2midi(pr, str(recreated_midi_path), samples_per_second=int(1/0.001))

# Try only with partitura

In [3]:
import partitura
from data_handling.io import pianoroll2midi, midi_to_3dpianoroll
midi_path = Path("/share/cp/projects/concept_composers/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_01_R1_2015_wav--3.midi")
npy_path = Path("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/pianoroll_ex.npy")
recreated_midi_path = ("/share/home/francesco/InvertibleCE/music_experiments/dataset_check/new_midi.mid")

In [4]:
def midi_to_3dpianoroll(midi_path, out_npy_path, frame_length, piano_range):
    part = partitura.load_performance_midi(midi_path)
    matrix1 = partitura.utils.compute_pianoroll(part,time_unit = "sec", time_div= int(1/frame_length), onset_only=True, piano_range=piano_range).toarray()
    matrix1[matrix1>0] = 1
    matrix2 = partitura.utils.compute_pianoroll(part,time_unit = "sec", time_div= int(1/frame_length), onset_only=False, piano_range=piano_range).toarray()
    pr3d = np.stack([matrix1,matrix2])
    pr3d = np.transpose(pr3d, (0,2,1))
    np.save(str(out_npy_path), pr3d)

In [6]:
midi_to_3dpianoroll(midi_path,npy_path, frame_length = 0.05, piano_range = True)
pr = np.transpose(np.load(npy_path), (0, 2, 1))
print(pr.shape)
pianoroll2midi(pr, str(recreated_midi_path), samples_per_second=int(1/0.05))

(2, 88, 4554)
